In [6]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import Calendar
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

class EditarStatusApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Editar Status dos Agendamentos")

        # Conectar ao banco de dados
        self.conn = self.connect_db()

        # Criar interface
        self.criar_interface()

    def connect_db(self):
        try:
            conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
            engine = create_engine(
                conexao_bd,
                connect_args={'options': '-csearch_path=sistema,public'}
            )
            conn = engine.connect()
            return conn
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return None

    def criar_interface(self):
        # Calendário de início
        tk.Label(self.root, text="Data Início").grid(row=0, column=0)
        self.cal_inicio = Calendar(self.root, selectmode="day", date_pattern="yyyy-mm-dd")
        self.cal_inicio.grid(row=0, column=1)

        # Calendário de fim
        tk.Label(self.root, text="Data Fim").grid(row=1, column=0)
        self.cal_fim = Calendar(self.root, selectmode="day", date_pattern="yyyy-mm-dd")
        self.cal_fim.grid(row=1, column=1)

        # Botão para buscar agendamentos
        btn_buscar = tk.Button(self.root, text="Buscar Agendamentos", command=self.buscar_agendamentos)
        btn_buscar.grid(row=2, column=0, columnspan=2)

        # Treeview para exibir os agendamentos
        self.tree = ttk.Treeview(self.root, columns=("ID", "Data", "Hora", "Cliente", "Médico", "Status"), show="headings")
        self.tree.grid(row=3, column=0, columnspan=2)
        for col in ["ID", "Data", "Hora", "Cliente", "Médico", "Status"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        # Scrollbar para a Treeview
        scrollbar = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.grid(row=3, column=2, sticky="ns")

        # Label e Combobox para alterar o status
        tk.Label(self.root, text="Novo Status").grid(row=4, column=0)
        self.combo_status = ttk.Combobox(self.root, values=["Agendado", "Cancelado", "Finalizado"], state="readonly")
        self.combo_status.grid(row=4, column=1)
        self.combo_status.current(0)

        # Botão para atualizar status
        btn_atualizar = tk.Button(self.root, text="Atualizar Status", command=self.atualizar_status)
        btn_atualizar.grid(row=5, column=0, columnspan=2)

        # Botão para deletar agendamento
        btn_deletar = tk.Button(self.root, text="Deletar Agendamento", command=self.deletar_agendamento)
        btn_deletar.grid(row=6, column=0, columnspan=2)

    def buscar_agendamentos(self):
        data_inicio = self.cal_inicio.get_date()
        data_fim = self.cal_fim.get_date()

        try:
            # Consultar agendamentos no período
            query = text("""
                SELECT A.ID_Agendamento, A.Data, A.Hora, P_Cliente.Nome as Cliente, P_Medico.Nome as Medico, A.Status
                FROM Agendamento A
                JOIN Cliente C ON A.ID_Cliente = C.ID_Cliente
                JOIN Pessoa P_Cliente ON C.ID_Pessoa = P_Cliente.ID_Pessoa
                JOIN Medico M ON A.ID_Medico = M.ID_Medico
                JOIN Pessoa P_Medico ON M.ID_Pessoa = P_Medico.ID_Pessoa
                WHERE A.Data BETWEEN :data_inicio AND :data_fim
                ORDER BY A.Data, A.Hora
            """)
            resultado = self.conn.execute(query, {
                "data_inicio": data_inicio,
                "data_fim": data_fim
            })

            # Limpar a Treeview
            self.tree.delete(*self.tree.get_children())

            # Inserir dados na Treeview formatados
            for row in resultado:
                # Formatar data e hora para exibição
                data_formatada = row[1].strftime("%d/%m/%Y")
                hora_formatada = row[2].strftime("%H:%M")
                self.tree.insert("", "end", values=(row[0], data_formatada, hora_formatada, row[3], row[4], row[5]))

        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao buscar agendamentos: {e}")

    def atualizar_status(self):
        # Obter item selecionado na Treeview
        item_selecionado = self.tree.selection()

        if not item_selecionado:
            messagebox.showerror("Erro", "Nenhum agendamento selecionado.")
            return

        # Obter o ID do agendamento (remover parênteses e converter para inteiro)
        agendamento_id = int(self.tree.item(item_selecionado[0], "values")[0])

        # Obter o novo status selecionado
        novo_status = self.combo_status.get()

        try:
            # Atualizar o status do agendamento no banco de dados
            update_query = text("""
                UPDATE Agendamento
                SET Status = :novo_status
                WHERE ID_Agendamento = :id_agendamento
            """)
            self.conn.execute(update_query, {
                "novo_status": novo_status,
                "id_agendamento": agendamento_id
            })

            # Confirmar a transação
            self.conn.commit()

            # Atualizar o status na Treeview
            self.tree.item(item_selecionado[0], values=(
                self.tree.item(item_selecionado[0], "values")[0],  # ID
                self.tree.item(item_selecionado[0], "values")[1],  # Data
                self.tree.item(item_selecionado[0], "values")[2],  # Hora
                self.tree.item(item_selecionado[0], "values")[3],  # Cliente
                self.tree.item(item_selecionado[0], "values")[4],  # Médico
                novo_status  # Status
            ))

            messagebox.showinfo("Sucesso", "Status atualizado com sucesso!")

        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao atualizar status: {e}")

    def deletar_agendamento(self):
        # Obter item selecionado na Treeview
        item_selecionado = self.tree.selection()

        if not item_selecionado:
            messagebox.showerror("Erro", "Nenhum agendamento selecionado.")
            return

        # Obter o ID do agendamento (remover parênteses e converter para inteiro)
        agendamento_id = int(self.tree.item(item_selecionado[0], "values")[0])

        try:
            # Deletar o agendamento no banco de dados
            delete_query = text("""
                DELETE FROM Agendamento
                WHERE ID_Agendamento = :id_agendamento
            """)
            self.conn.execute(delete_query, {
                "id_agendamento": agendamento_id
            })

            # Confirmar a transação
            self.conn.commit()

            # Remover o item da Treeview
            self.tree.delete(item_selecionado[0])

            messagebox.showinfo("Sucesso", "Agendamento deletado com sucesso!")

        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao deletar agendamento: {e}")

# Inicialização da interface gráfica
if __name__ == "__main__":
    root = tk.Tk()
    app = EditarStatusApp(root)
    root.mainloop()
